In [125]:
import pandas as pd
import numpy as np
import random

In [126]:
df = pd.read_csv(r'./data/preprocess.csv')

df.head()

,IDX,mean_ndvi_2022-06-02,mean_ndvi_2022-06-17,mean_ndvi_2022-06-07,mean_ndvi_2022-06-22,mean_ndvi_2022-06-12,mean_ndre_2022-06-02,mean_ndre_2022-06-17,mean_ndre_2022-06-07,mean_ndre_2022-06-12,IDX.1,sum_ndvi_2022-06-02,sum_ndvi_2022-06-17,sum_ndvi_2022-06-07,sum_ndvi_2022-06-22,sum_ndvi_2022-06-12,sum_ndre_2022-06-02,sum_ndre_2022-06-17,sum_ndre_2022-06-07,sum_ndre_2022-06-12
0,3942,0.007476,0.212770,0.180844,0.038478,0.219896,-0.017073,0.151714,0.123746,0.153333,3942,0.530825,15.106694,12.839933,2.731913,15.612608,-1.212181,10.771678,8.785966,10.886635
1,4053,0.008709,0.202058,0.165046,-0.019584,0.198812,-0.007738,0.134886,0.106043,0.128798,4053,1.689533,39.199349,32.018860,-3.799293,38.569600,-1.501124,26.167940,20.572346,24.986805
2,4196,0.204576,0.192939,0.156745,-0.011448,0.186642,0.226347,0.128658,0.095656,0.117090,4196,19.639275,18.522111,15.047543,-1.099016,17.917662,21.729303,12.351145,9.183014,11.240659
3,4422,-0.001838,0.198717,0.181343,-0.002894,0.215428,-0.036878,0.132829,0.113879,0.136089,4422,-0.104746,11.326846,10.336535,-0.164956,12.279409,-2.102046,7.571269,6.491099,7.757051
4,4424,-0.046762,0.205482,0.185708,-0.005377,0.218237,-0.094255,0.145178,0.124556,0.148208,4424,-1.730197,7.602816,6.871180,-0.198932,8.074769,-3.487440,5.371600,4.608560,5.483689


In [127]:
target = pd.read_csv('./data/target_PROD.csv')

In [128]:
l = target[['PROD']]

str(list(l['PROD']))

'[31595, 3285, 2581, 13155, 2487, 0, 3843, 2537, 14912, 10374, 5482, 6076, 4489, 1235, 12240, 10599, 4426, 691, 5631, 2173, 1729, 12134, 3248, 4940, 4322]'

In [129]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = pd.read_csv(r'./data/preprocess.csv')
y = target[['PROD']]

X_train, X_test, y_train, y_test= train_test_split(
    
    X,y,
    train_size   = 0.8,
    random_state = 1234,
    shuffle      = True
)

En este cuaderno probamos diferentes modelos de regresión lineal junto con una máquina-vector-soporte para regresión. Intentaremos incrementar los resultados de los modelos a través de métodos de boosting. Hacemos uso de técnicas de búsqueda de hyperparámetros añadidas a los enesembles.

## SVR _ best_params

In [130]:
from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVR

svm_reg = SVR()

In [131]:
param_grid = [
    {'kernel': ['linear'], 'C': [1.0, 3.0]},
    {'kernel': ['rbf'], 'C': [1.0, 3.0, 5.0], 'gamma': [0.01, 0.1, 1.0]} ]

grid_search_RF = GridSearchCV(

    svm_reg, 
    param_grid, cv=3,
    scoring='neg_mean_squared_error',
    refit= True, verbose=2, n_jobs=-1)

grid_search_RF.fit(X_train, y_train)

Fitting 3 folds for each of 11 candidates, totalling 33 fits


GridSearchCV(cv=3, estimator=SVR(), n_jobs=-1,
             param_grid=[{'C': [1.0, 3.0], 'kernel': ['linear']},
                         {'C': [1.0, 3.0, 5.0], 'gamma': [0.01, 0.1, 1.0],
                          'kernel': ['rbf']}],
             scoring='neg_mean_squared_error', verbose=2)

In [132]:
negative_mse = grid_search_RF.best_score_
rmse = np.sqrt(-negative_mse)
rmse

7190.328721761205

In [133]:
grid_search_RF.best_params_

{'C': 1.0, 'gamma': 0.1, 'kernel': 'rbf'}

In [134]:
cvres = grid_search_RF.cv_results_

# representamos el valor para todos los hiperparámetros

for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):

    print(np.sqrt(-mean_score), params)

9106.381709964893 {'C': 1.0, 'kernel': 'linear'}
8978.37831465298 {'C': 3.0, 'kernel': 'linear'}
7190.328722010281 {'C': 1.0, 'gamma': 0.01, 'kernel': 'rbf'}
7190.328721761205 {'C': 1.0, 'gamma': 0.1, 'kernel': 'rbf'}
7190.328721761205 {'C': 1.0, 'gamma': 1.0, 'kernel': 'rbf'}
7190.328722508436 {'C': 3.0, 'gamma': 0.01, 'kernel': 'rbf'}
7190.328721761205 {'C': 3.0, 'gamma': 0.1, 'kernel': 'rbf'}
7190.328721761205 {'C': 3.0, 'gamma': 1.0, 'kernel': 'rbf'}
7190.328723006588 {'C': 5.0, 'gamma': 0.01, 'kernel': 'rbf'}
7190.328721761205 {'C': 5.0, 'gamma': 0.1, 'kernel': 'rbf'}
7190.328721761205 {'C': 5.0, 'gamma': 1.0, 'kernel': 'rbf'}


In [135]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error

ada_reg_svm = AdaBoostRegressor(
    
      SVR(**grid_search_RF.best_params_),
) 

In [136]:
param_grid = {
    'n_estimators': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 20],
    'learning_rate': [(0.97 + x / 100) for x in range(0, 8)],
}

grid_search_ADA_svm = GridSearchCV(

    ada_reg_svm, 
    
    param_grid, cv=3,
    scoring='neg_mean_squared_error',
    refit= True, verbose=2, n_jobs=-1)

grid_search_ADA_svm.fit(X_train, y_train)

Fitting 3 folds for each of 96 candidates, totalling 288 fits


GridSearchCV(cv=3, estimator=AdaBoostRegressor(base_estimator=SVR(gamma=0.1)),
             n_jobs=-1,
             param_grid={'learning_rate': [0.97, 0.98, 0.99, 1.0, 1.01, 1.02,
                                           1.03, 1.04],
                         'n_estimators': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          20]},
             scoring='neg_mean_squared_error', verbose=2)

In [137]:
grid_search_ADA_svm.best_params_

{'learning_rate': 1.01, 'n_estimators': 9}

In [139]:

y_pred = grid_search_ADA_svm.predict(X_test)

ada_svr_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))

y_pred = grid_search_ADA_svm.predict(X_train)

ada_svr_rmse_train = np.sqrt(mean_squared_error(y_train, y_pred))

## LINEAR + ADA_BOOST

In [140]:
# AdaBoost Regressor, para enlazar varios modelos vamos a probar con alguno de los modelos lineales y así poder incluir todos en el AdaBoostRegressor.feature_importances_

# LINEALES:

import warnings

warnings.filterwarnings('ignore')

In [141]:
# list of alpha to tune

params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000 ]}


In [142]:
# defining cross validation folds as 5
folds = 5

### ADA_+_LASSO

In [143]:
from sklearn.linear_model import Lasso

lasso = Lasso()

grid_cv_lasso = GridSearchCV(

    estimator=lasso,

    param_grid=params,
    scoring='neg_mean_absolute_error',
    cv=folds,
    return_train_score=True,
    verbose=1)
    
#fiting model_cv
grid_cv_lasso.fit(X_train,y_train)

Fitting 5 folds for each of 28 candidates, totalling 140 fits


GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3,
                                   0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0,
                                   4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50,
                                   100, 500, 1000]},
             return_train_score=True, scoring='neg_mean_absolute_error',
             verbose=1)

In [144]:
grid_cv_lasso.best_params_

{'alpha': 1000}

In [145]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error

ada_reg_lasso = AdaBoostRegressor(
    
      Lasso(**grid_cv_lasso.best_params_),
) 


In [146]:
param_grid = {
    'n_estimators': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 20],
    'learning_rate': [(0.97 + x / 100) for x in range(0, 8)],
}

grid_search_ADA_lasso= GridSearchCV(

    ada_reg_lasso, 
    
    param_grid, cv=3,
    scoring='neg_mean_squared_error',
    refit= True, verbose=2, n_jobs=-1)

grid_search_ADA_lasso.fit(X_train, y_train)

Fitting 3 folds for each of 96 candidates, totalling 288 fits


GridSearchCV(cv=3,
             estimator=AdaBoostRegressor(base_estimator=Lasso(alpha=1000)),
             n_jobs=-1,
             param_grid={'learning_rate': [0.97, 0.98, 0.99, 1.0, 1.01, 1.02,
                                           1.03, 1.04],
                         'n_estimators': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          20]},
             scoring='neg_mean_squared_error', verbose=2)

In [147]:
grid_search_ADA_lasso.best_params_

{'learning_rate': 1.02, 'n_estimators': 4}

In [148]:
y_pred = grid_search_ADA_lasso.predict(X_test)

ada_lasso_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))

y_pred = grid_search_ADA_lasso.predict(X_train)

ada_lasso_rmse_train = np.sqrt(mean_squared_error(y_train, y_pred))

### ADA_+_RIDGE

In [149]:
from sklearn.linear_model import Ridge

ridge = Ridge()

grid_cv_ridge = GridSearchCV(

    estimator=ridge,

    param_grid=params,
    scoring='neg_mean_absolute_error',
    cv=folds,
    return_train_score=True,
    verbose=1)
    
grid_cv_ridge.fit(X_train,y_train)

Fitting 5 folds for each of 28 candidates, totalling 140 fits


GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3,
                                   0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0,
                                   4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50,
                                   100, 500, 1000]},
             return_train_score=True, scoring='neg_mean_absolute_error',
             verbose=1)

In [150]:
grid_cv_ridge.best_params_

{'alpha': 1000}

In [151]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error

ada_reg_ridge = AdaBoostRegressor(
    
      Ridge(**grid_cv_ridge.best_params_),
)


In [152]:
param_grid = {
    'n_estimators': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 20],
    'learning_rate': [(0.97 + x / 100) for x in range(0, 8)],
}

grid_search_ADA_ridge= GridSearchCV(

    ada_reg_ridge, 
    
    param_grid, cv=3,
    scoring='neg_mean_squared_error',
    refit= True, verbose=2, n_jobs=-1)

grid_search_ADA_ridge.fit(X_train, y_train)

Fitting 3 folds for each of 96 candidates, totalling 288 fits


GridSearchCV(cv=3,
             estimator=AdaBoostRegressor(base_estimator=Ridge(alpha=1000)),
             n_jobs=-1,
             param_grid={'learning_rate': [0.97, 0.98, 0.99, 1.0, 1.01, 1.02,
                                           1.03, 1.04],
                         'n_estimators': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          20]},
             scoring='neg_mean_squared_error', verbose=2)

In [153]:
y_pred = grid_search_ADA_ridge.predict(X_test)

ada_ridge_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))

y_pred = grid_search_ADA_ridge.predict(X_train)

ada_ridge_rmse_train = np.sqrt(mean_squared_error(y_train, y_pred))

In [154]:
from sklearn.linear_model import ElasticNet

elastic = ElasticNet()

grid_cv_elastic = GridSearchCV(

    estimator=elastic,

    param_grid=params,
    scoring='neg_mean_absolute_error',
    cv=folds,
    return_train_score=True,
    verbose=1)
    
grid_cv_elastic.fit(X_train,y_train)

Fitting 5 folds for each of 28 candidates, totalling 140 fits


GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3,
                                   0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0,
                                   4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50,
                                   100, 500, 1000]},
             return_train_score=True, scoring='neg_mean_absolute_error',
             verbose=1)

In [155]:
grid_cv_elastic.best_params_ 

{'alpha': 500}

In [156]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error

ada_reg_elastic = AdaBoostRegressor(
    
      ElasticNet(**grid_cv_elastic.best_params_),
) 


In [157]:
param_grid = {
    'n_estimators': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 20],
    'learning_rate': [(0.97 + x / 100) for x in range(0, 8)],
}

grid_search_ADA_elastic = GridSearchCV(

    ada_reg_svm, 
    
    param_grid, cv=3,
    scoring='neg_mean_squared_error',
    refit= True, verbose=2, n_jobs=-1)

grid_search_ADA_elastic.fit(X_train, y_train)


Fitting 3 folds for each of 96 candidates, totalling 288 fits


GridSearchCV(cv=3, estimator=AdaBoostRegressor(base_estimator=SVR(gamma=0.1)),
             n_jobs=-1,
             param_grid={'learning_rate': [0.97, 0.98, 0.99, 1.0, 1.01, 1.02,
                                           1.03, 1.04],
                         'n_estimators': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          20]},
             scoring='neg_mean_squared_error', verbose=2)

In [158]:
grid_search_ADA_elastic.best_params_ 

{'learning_rate': 1.02, 'n_estimators': 9}

In [159]:
y_pred = grid_search_ADA_elastic.predict(X_test)

ada_elastic_rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))

y_pred = grid_search_ADA_elastic.predict(X_train)

ada_elastic_rmse_train = np.sqrt(mean_squared_error(y_train, y_pred))

## COMPARE _ test_res _ LINEAR / SVR

In [160]:
# añadimos a un df los resultados obtenidos para poder analizar correctamente los resultados.

In [161]:
def append_row(df, row):
    
    return pd.concat([
                df, 
                pd.DataFrame([row], columns=row.index)]
           ).reset_index(drop=True)

In [162]:
d = {'SVR', 'LASSO', 'RIDGE', 'ELASTIC'}

df = pd.DataFrame(columns=d)

row_train = pd.Series({'SVR':ada_svr_rmse_train, 'LASSO':ada_lasso_rmse_train, 'RIDGE':ada_ridge_rmse_train, 'ELASTIC':ada_elastic_rmse_train})
row_test = pd.Series({'SVR':ada_svr_rmse_test, 'LASSO':ada_lasso_rmse_test, 'RIDGE':ada_ridge_rmse_test, 'ELASTIC':ada_elastic_rmse_test})

df = append_row(df, row_train)
df = append_row(df, row_test)

df

,RIDGE,SVR,LASSO,ELASTIC
0,6743.320150,7684.064797,7284.417075,7099.151791
1,6325.124421,4277.029779,5657.553874,4090.361671


In [163]:
# conc:

# después de haber usado varios modelos, utilizar búsqueda de hyperparámetros, incluir ADABOOST; vemos que no mejoran o mantienen los resultados del RandomForest o la red neuronal.
# puede que en alguno de estos casos se produzca algo de sobreajuste a los datos.